# Read tokenized txt and conll

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize


def align_span_and_token(text_path):
    text = None
    with open(text_path, "r") as fr:
        text = fr.read()
#     span_and_token = []
    tokenized_sent = [word_tokenize(sent) for sent in sent_tokenize(text)]
    
    span_and_token = []
    span_start_index = 0
    token_index = 0
#     print(tokenized_sent)
    for sent in tokenized_sent:
        for token in sent:
#             print(token)
            find_start = False
            while find_start == False:
                if text[span_start_index: span_start_index+1] == token[0]:
                    find_start = True
                if find_start == False: 
                    span_start_index+=1
                    continue
                find_token = False
                for span_end_index in range(span_start_index+1, len(text)+1):
                    if str(text[span_start_index: span_end_index]) == str(token):
                        span_and_token.append([span_start_index, span_end_index, token, token_index])
                        token_index+=1
                        span_start_index = span_end_index
                        find_token = True
                        break
                if find_token == False:
                    print("error. can not find the tokne.", token, span_start_index)
            
            
            
        span_start_index+=1
        span_and_token.append([]) # [] means the end of the sentence

    
#     print(span_and_token)
    span_and_token_count = 0
    for s_a_t in span_and_token:
        if s_a_t != []:
            span_and_token_count+=1
    assert(span_and_token_count == sum([len(token) for token in tokenized_sent]))

    return span_and_token  # [span_index_start, span_index_end, txt, token_id]


In [ ]:
def process_ann(file_name):
    """ read the file of ann and matain entity label and relationship """
    mentions = {}
    mentions_span = {}
#     mentions_type = {}
    relations = {}
#     relation_type = ["Coreference", "Transformed", "Reaction-associated", "Work-up", "Contained"]
    relation_type = ["Coreference", "Transformed", "Ingredient_without-state-change_associated",\
                     "Ingredient_with-state-change_associated"]
    for r_type in relation_type:
        relations[r_type] = []
        
    with open(file_name, "r") as fr:
        lines = fr.readlines()
        for line in lines:
#             print(line)
            ###filter out some unrelated label from NER
            split_line = line.split("\t")
            label = split_line[1].split(" ")[0]

            ## mention
            if "T" in split_line[0]:
                ## text
                mention_text = split_line[2].split("\n")[0]
                mentions[split_line[0]] = mention_text

                ### span
                span = split_line[1].split(";")
#               discontious mention:
                if len(span)> 1: 
                    mentions_span[split_line[0]] = []
                    for sp in span:
                        s = sp.split(" ")
                        mentions_span[split_line[0]].append((int(s[-2]), int(s[-1])))
#                     contious mention:
                else:
                    span = split_line[1].split(" ")
                    mentions_span[split_line[0]] = [(int(span[1]), int(span[2]))]
#           
                ### mention_type 
#                     print(split_line)
#                 ty = split_line[1].split(" ")[0]
#                 mentions_type[ann_file][split_line[0]] = ty

#             R3	Transformed Arg1:T13 Arg2:T5	
            ## relation    
            if "R" in split_line[0]:
                r_element = split_line[1].split(" ")
                r_type = r_element[0]
                r_anaphor_t = r_element[1].split(":")[1]
                r_antecedent_t = r_element[2].split(":")[1]
                relations[r_type].append(list([r_anaphor_t, r_antecedent_t]))

    return mentions, mentions_span, relations

In [ ]:
# mentions, mentions_span, relations = process_ann("./annotated_data/101115.ann")
# mentions, mentions_span, relations

In [ ]:
def transfer_spanID_to_tokenID(lines, mentions_span):
    mentions_token = {}
    for t_key in mentions_span.keys():
        spans = mentions_span[t_key]
        # maybe discontinuous mention 
        
        token_temp = []
        for s in spans:
#             print(s)
            start_t = None
            end_t = None
            
            start_s = s[0]
            end_s = s[1]
            
            find_start_id = False
            find_end_id = False
            
            for line in lines: 
                if line == []:
                    continue
                if line[0] == start_s:
                    find_start_id = True
                    start_t = line[3]
                if line[1] == end_s: 
                    find_end_id = True
                    end_t = line[3]
                
                if find_start_id and find_end_id: 
                    break
            
            
            if find_start_id and not find_end_id:
                print("A: something wrong with the ID matching - only find the start of span")
                print("spans:", spans)
                print("span_token: ", lines)
                end_t = start_t
                print("then get token: ", [start_t, end_t])
                
            if not find_start_id and find_end_id:
                print("A: something wrong with the ID matching - only find the end of span")
                print("spans:", spans)
                print("span_token: ", lines)
                start_t = end_t                
                print("then get token: ", [start_t, end_t])
                
            if not find_start_id and not find_end_id: # get the brodar span:
                for line_index in range(len(lines)-1):
                    
                    next_index = line_index +1 
                    if lines[line_index] == []:
                        continue
                    if lines[line_index+1] == []:
                        next_index = line_index

                    if lines[line_index][0] <= start_s and lines[next_index][0] >= start_s:
                        find_start_id = True
                        start_t = lines[line_index][3]
                        
                    if lines[line_index][1] <= end_s and lines[next_index][1] >= end_s: 
                        find_end_id = True
                        end_t = lines[next_index][3]

                    if find_start_id and find_end_id: 
                        break
                    
            if not find_start_id or not find_end_id: # still nothing so we discard them      
                continue
                
            if start_t > end_t:
                print("b: something wrong with the ID matching - token: start is larger than end")
#             print([start_t, end_t])   
            token_temp.append([start_t, end_t])   
        mentions_token[t_key]= token_temp

    return mentions_token

In [ ]:
import json

def constcut_jsonline_format(doc_id, lines, mentions_token, relations):
    # get sentences 
    instance = {}
#     Index(['speakers', 'doc_key', 'sentences', 'constituents', 'clusters', 'ner'], dtype='object')
    instance["doc_key"] = doc_id
    
    sentences = []
    sentence = []
    for line in lines:
        if line == []:
            sentences.append(sentence)
            sentence = []
            continue
        else:
            sentence.append(line[2])
#     relations_types = relations.keys() # coref, TR, RA, WU, CA

    instance["sentences"] = sentences
    
    #useless keys
    # speaker
#     instance["speakers"] = []
#     for s in sentences:
#         instance["speakers"].append(["-"]*len(s))
#     instance["constituents"] = []
#     instance["ner"] = []
    
    # relation keys
    relations_token = {}
    for r_type in relations:
        relations_token[r_type] = []
        for anphor_antecedent_pair in relations[r_type]:
            relations_token[r_type].append([mentions_token[anphor_antecedent_pair[0]],mentions_token[anphor_antecedent_pair[1]]])
        
        instance[r_type] = relations_token[r_type]

    return instance

In [ ]:
import os 

revise = True

root_path = "/home/biaoyan/data/Recipe/RecipeRef/"

data_type = "train"

if not revise:

    annotation_path = "%s/%s/"%(root_path, data_type)
else: 
    annotation_path = "%s/%s_no_state/"%(root_path, data_type)


    
annotation_file_name = os.listdir(annotation_path)
annotation_ann_name = []
annotation_txt_name = []
annotation_doc_id = []

for file in annotation_file_name:
    if ".ann" in file:
        annotation_ann_name.append(file)
        annotation_doc_id.append(file.split(".")[0])        
    elif ".txt" in file:
        annotation_txt_name.append(file)
annotation_doc_id = sorted(annotation_doc_id)

In [ ]:
len(annotation_doc_id)

In [ ]:
import json
for doc_id in annotation_doc_id:
    print(doc_id)
    if not revise: 
        jsonlines_path_part = "%s/%s_converted_jsonlines/"%(root_path, data_type)
    else:
        jsonlines_path_part = "%s/%s_converted_jsonlines_no_state/"%(root_path, data_type)
    
    
    if not os.path.exists(jsonlines_path_part):
        os.makedirs(jsonlines_path_part)
        
    jsonlines_path = jsonlines_path_part+doc_id+".jsonlines" 
    
        
    with open(jsonlines_path, 'w') as fw:
        txt_path = annotation_path+doc_id+".txt"
#         tokenized_conll_path = "./chemical-patents/final_anaphora_data/snippets_only_token/"+doc_id+".conll"
#         lines = align_span_and_token_from_tokenized_conll(txt_path, tokenized_conll_path)
        lines = align_span_and_token(txt_path)
        ann_path = annotation_path+doc_id+".ann"
        mentions, mentions_span, relations = process_ann(ann_path)
        mentions_token = transfer_spanID_to_tokenID(lines, mentions_span)
        instance = constcut_jsonline_format(doc_id, lines, mentions_token, relations)

    # acttually, we should dump this jonsonlines 
        json.dump(instance, fw)
        fw.write("\n")
#     instances.append(instance)
# annotation_doc_id = sorted(annotation_doc_id)
print("Done!!!")

In [ ]:
import os 
def merge_jsonlines(jsonlines_path, merge_jsonlines_path):

    annotation_doc_id = os.listdir(merge_jsonlines_path)

    with open(jsonlines_path, 'w') as fw:
        for doc_path in annotation_doc_id:
            jsonlines_path_part = merge_jsonlines_path + doc_path  
            with open(jsonlines_path_part, "r") as fr:
                fw.write(fr.read())
                
if not revise: 
    jsonlines_path = "%s/%s.english.jsonlines"%(root_path, data_type)
else:
    jsonlines_path = "%s/%s_no_state.english.jsonlines"%(root_path, data_type)
    
# jsonlines_path = 
merge_jsonlines_path = jsonlines_path_part


merge_jsonlines(jsonlines_path, merge_jsonlines_path)
# get_partition(data_type, annotation_path, annotation_doc_id, start_t, end_t)
